# Generation of learning database from a stochastic reactors simulation

In [1]:
from ai_reacting_flows.stochastic_reactors_data_gen.database_processing import LearningDatabase

In [2]:
# Dictionary to store data processing parameters
dtb_processing_parameters = {}

dtb_processing_parameters["dtb_folder"] = "../scripts/STOCH_DTB_hotspot_H2_DEV"
dtb_processing_parameters["database_name"] = "database_1"
dtb_processing_parameters["log_transform"] = True
dtb_processing_parameters["threshold"] = 1.0e-10
dtb_processing_parameters["output_omegas"] = True

In [6]:
database = LearningDatabase(dtb_processing_parameters)

>> Processed dataset database_1 already exists => deleting


In [7]:
database.clusterize_dataset("kmeans", 3)
# database.clusterize_dataset("progvar", 2, c_bounds=[0,0.5,1.0])

>> Performing k-means clustering


In [8]:
database.process_database()